In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [7]:

#Load all datasets
data_path = '/Users/poornavenkat/Documents/GitHub/SQL-Projects/02-Ecommerce-Customer-Analytics/data/Raw/'

orders = pd.read_csv(data_path + 'olist_orders_dataset.csv')
customers = pd.read_csv(data_path + 'olist_customers_dataset.csv')
order_items = pd.read_csv(data_path + 'olist_order_items_dataset.csv')
payments = pd.read_csv(data_path + 'olist_order_payments_dataset.csv')
reviews = pd.read_csv(data_path + 'olist_order_reviews_dataset.csv')

print("Datasets loaded successfully!")
print(f"Orders: {orders.shape}")
print(f"Customers: {customers.shape}")
print(f"Order Items: {order_items.shape}")
print(f"Payments: {payments.shape}")
print(f"Reviews: {reviews.shape}")


Datasets loaded successfully!
Orders: (99441, 8)
Customers: (99441, 5)
Order Items: (112650, 7)
Payments: (103886, 5)
Reviews: (99224, 7)


In [8]:
#Explore the orders dataset
print("Orders Dataset Info:")
print(orders.info())
print("\nFirst 5 rows:")
orders.head()

Orders Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None

First 5 rows:


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [9]:
#Check for missing values
def check_missing_values(df, dataset_name):
    missing = df.isnull().sum()
    missing_percent = 100 * missing / len(df)
    missing_table = pd.DataFrame({
        'Missing_Count': missing,
        'Percentage': missing_percent
    })
    missing_table = missing_table[missing_table['Missing_Count'] > 0].sort_values('Percentage', ascending=False)
    print(f"\n{dataset_name} - Missing Values:")
    print(missing_table)
    return missing_table

check_missing_values(orders, 'Orders')
check_missing_values(customers, 'Customers')
check_missing_values(payments, 'Payments')


Orders - Missing Values:
                               Missing_Count  Percentage
order_delivered_customer_date           2965    2.981668
order_delivered_carrier_date            1783    1.793023
order_approved_at                        160    0.160899

Customers - Missing Values:
Empty DataFrame
Columns: [Missing_Count, Percentage]
Index: []

Payments - Missing Values:
Empty DataFrame
Columns: [Missing_Count, Percentage]
Index: []


,Missing_Count,Percentage


In [10]:
# Fix datetime columns
datetime_columns = ['order_purchase_timestamp', 'order_approved_at', 
                   'order_delivered_carrier_date', 'order_delivered_customer_date',
                   'order_estimated_delivery_date']

for col in datetime_columns:
    orders[col] = pd.to_datetime(orders[col])

print("DateTime columns converted successfully!")

DateTime columns converted successfully!


In [13]:
# Create master dataset by merging
master_df = orders.merge(customers, on='customer_id', how='left')

master_df = master_df.merge(
    order_items.groupby('order_id').agg({
        'product_id': 'count',
        'price': 'sum',
        'freight_value': 'sum'
    }).rename(columns={
        'product_id': 'item_count',
        'price': 'total_price',
        'freight_value': 'total_freight'
    }),
    on='order_id',
    how='left'
)

master_df = master_df.merge(
    payments.groupby('order_id')['payment_value'].sum().rename('payment_total'),
    on='order_id',
    how='left'
)

print(f"Master dataset created: {master_df.shape}")
master_df.head()

# ---- Save Master Dataset ----
master_df.to_csv('/Users/poornavenkat/Documents/GitHub/SQL-Projects/02-Ecommerce-Customer-Analytics/data/master_dataset.csv',
                 index=False)

print("Master dataset saved successfully!")


Master dataset created: (99441, 17)
Master dataset saved successfully!


In [12]:
# Basic statistics
print("Order Status Distribution:")
print(orders['order_status'].value_counts())
print("\n" + "="*50)

print("\nOrders by Year-Month:")
orders['year_month'] = orders['order_purchase_timestamp'].dt.to_period('M')
print(orders['year_month'].value_counts().sort_index())

Order Status Distribution:
order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64


Orders by Year-Month:
year_month
2016-09       4
2016-10     324
2016-12       1
2017-01     800
2017-02    1780
2017-03    2682
2017-04    2404
2017-05    3700
2017-06    3245
2017-07    4026
2017-08    4331
2017-09    4285
2017-10    4631
2017-11    7544
2017-12    5673
2018-01    7269
2018-02    6728
2018-03    7211
2018-04    6939
2018-05    6873
2018-06    6167
2018-07    6292
2018-08    6512
2018-09      16
2018-10       4
Freq: M, Name: count, dtype: int64
